Apple: AAPL
NVIDIA: NVDA
Microsoft: MSFT
Tesla: TSLA
Amazon: AMZN
Alphabet: GOOGL (Class A) / GOOG (Class C)
Meta: META

In [25]:
df = fdr.DataReader('AAPL', '2014-01-01')
df

,Open,High,Low,Close,Volume,Adj Close
2013-12-31,19.791786,20.045713,19.785713,20.036428,223084400,17.480125
2014-01-02,19.845715,19.893929,19.715000,19.754642,234684800,17.234297
2014-01-03,19.745001,19.775000,19.301071,19.320715,392467600,16.855728
2014-01-06,19.194643,19.528570,19.057142,19.426071,412610800,16.947638
2014-01-07,19.440001,19.498571,19.211430,19.287144,317209200,16.826450
...,...,...,...,...,...,...
2024-12-02,237.270004,240.789993,237.160004,239.589996,48137100,239.589996
2024-12-03,239.809998,242.759995,238.899994,242.649994,38861000,242.649994
2024-12-04,242.869995,244.110001,241.250000,243.009995,44383900,243.009995
2024-12-05,243.990005,244.539993,242.130005,243.039993,40033900,243.039993


In [7]:
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import numpy as np

# 1. 종목 데이터 가져오기
stocks = {
    'Apple': fdr.DataReader('AAPL', '2014-01-01'),
    'Nvidia': fdr.DataReader('NVDA', '2014-01-01'),
    'Microsoft': fdr.DataReader('MSFT', '2014-01-01'),
    'Tesla': fdr.DataReader('TSLA', '2014-01-01'),
    'Amazon': fdr.DataReader('AMZN', '2014-01-01'),
    'Alphabet A': fdr.DataReader('GOOGL', '2014-01-01'),
    'Alphabet C': fdr.DataReader('GOOG', '2014-01-01'),
    'Meta': fdr.DataReader('META', '2014-01-01')
}

# 2. Yahoo Finance에서 PER, PBR, ROE, ROA 크롤링
tickers = {
    'Apple': 'AAPL',
    'Nvidia': 'NVDA',
    'Microsoft': 'MSFT',
    'Tesla': 'TSLA',
    'Amazon': 'AMZN',
    'Alphabet A': 'GOOGL',
    'Alphabet C': 'GOOG',
    'Meta': 'META'
}

financial_data = {}

# 3. 각 종목의 PER, PBR, ROE, ROA 가져오기
for company, ticker in tickers.items():
    stock = yf.Ticker(ticker)
    try:
        # PER, PBR, ROE, ROA 데이터 가져오기
        per = stock.info.get('trailingPE')  # PER 데이터
        roe = stock.info.get('returnOnEquity')  # ROE 데이터
        roa = stock.info.get('returnOnAssets')  # ROA 데이터
        pbr = stock.info.get('priceToBook')  # PBR 데이터
        
        # 데이터가 없는 경우 처리
        if per is None:
            per = "N/A"
        if roe is None:
            roe = "N/A"
        else:
            roe *= 100  # ROE를 %로 변환
        if roa is None:
            roa = "N/A"
        else:
            roa *= 100  # ROA를 %로 변환
        if pbr is None:
            pbr = "N/A"

        financial_data[company] = {'PER': per, 'PBR': pbr, 'ROE': roe, 'ROA': roa}
    except Exception as e:
        print(f"Error fetching financial data for {company} ({ticker}): {e}")

# 4. 전체 그래프 설정 (1개의 이미지에 모든 종목의 그래프를 포함)
plt.figure(figsize=(20, 20))  # 전체 그림 크기 설정 (가로 20인치, 세로 20인치)

# 5. 각 종목별 그래프 추가
for i, (stock_name, df) in enumerate(stocks.items()):
    # 일간 수익률 계산
    df['daily_return'] = df['Close'].pct_change()
    
    # 누적 수익률 계산
    df['cumulative_return'] = (1 + df['daily_return']).cumprod()
    
    # 변동성 계산 (표준편차 * sqrt(252))
    volatility = df['daily_return'].std() * np.sqrt(252) * 100
    
    # 각 종목의 그래프를 subplot에 추가
    plt.subplot(4, 2, i + 1)  # 4행 2열로 subplot 배치
    plt.plot(df.index, df['cumulative_return'], label=f'Cumulative Return ({stock_name})', color='blue')
    
    # 누적 수익률, ROE, ROA, PER, PBR 및 변동성 표시
    final_return = df['cumulative_return'].iloc[-1]
    plt.text(0.05, 0.90, f'Return: {final_return*100:.2f}%', 
             transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', color='black')
    plt.text(0.05, 0.85, f'ROE: {financial_data[stock_name]["ROE"]}', 
             transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', color='purple')
    plt.text(0.05, 0.80, f'ROA: {financial_data[stock_name]["ROA"]}', 
             transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', color='darkorange')
    plt.text(0.05, 0.75, f'PER: {financial_data[stock_name]["PER"]}', 
             transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', color='green')
    plt.text(0.05, 0.70, f'PBR: {financial_data[stock_name]["PBR"]}', 
             transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', color='blue')
    plt.text(0.05, 0.65, f'Volatility: {volatility:.4f}%', 
             transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', color='red')
    
    # 그래프 제목 및 축 설정
    plt.title(f'{stock_name} - Cumulative Return', fontsize=12)
    plt.xlabel('Date', fontsize=10)
    plt.ylabel('Cumulative Return', fontsize=10)
    plt.legend(fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)

# 6. 그래프 레이아웃 조정
plt.tight_layout()

# 7. 전체 그래프를 이미지로 저장 (JPG 형식)
plt.savefig('Magnificent_Seven.jpg', format='jpg')

# 8. 그래프 닫기
plt.close()

print("Saved the combined graph as 'Magnificent_Seven.jpg'")

Saved the combined graph as 'Magnificent_Seven.jpg'
